In [1]:
import requests
import pandas as pd

# Base URL for FAOSTAT API
BASE_URL = "https://api.fao.org/data/1.0/data/"

def fetch_faostat_data(domain, params, dataset_name):
    url = f"{BASE_URL}{domain}"
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()  
        df = pd.DataFrame(data.get("data", []))
        df.to_csv(f"{dataset_name}.csv", index=False)
        return df
    else:
        print(f"Error fetching {dataset_name}: {response.status_code}")
        return pd.DataFrame()

In [2]:
params_security = {
    "area": "all", 
    "year": "2015-2020",
    "item": "Calories per capita",
}
food_security_df = fetch_faostat_data(domain="FB", params=params_security, dataset_name="Food_Security_Indicators")


Error fetching Food_Security_Indicators: 404
